In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tabular-playground-series-nov-2021/sample_submission.csv
/kaggle/input/tabular-playground-series-nov-2021/train.csv
/kaggle/input/tabular-playground-series-nov-2021/test.csv


In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import gc
import tensorflow as tf

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

from xgboost import XGBClassifier

warnings.filterwarnings("ignore")

In [3]:
# https://www.tensorflow.org/tutorials/keras/keras_tuner
# https://keras.io/keras_tuner/
# https://keras.io/guides/keras_tuner/getting_started/

!pip install -q -U keras-tuner

In [4]:
def evaluate_model(model, x, y):
    y_pred_prob = model.predict_proba(x)[:, 1]
    auc_roc = roc_auc_score(y, y_pred_prob)
    return {'auc_roc_curve' : auc_roc}

In [5]:
seed = 47

In [6]:
train_df = pd.read_csv('/kaggle/input/tabular-playground-series-nov-2021/train.csv', sep=',')

# XGBoost Baseline

In [7]:
x_train = train_df.drop(['id', 'target'], axis=1).values
y_train = train_df['target'].values 
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size = 0.2, random_state = seed)

In [8]:
params = {'colsample_bytree': 0.1,
          'eta': 0.12,
          'gamma': 5, 
          'max_depth': 2,
          'min_child_weight': 9,
          'n_estimators': 11000, 
          'subsample': 0.9}

model = XGBClassifier(**params, 
                      random_state=seed, 
                      tree_method='gpu_hist', 
                      predictor='gpu_predictor',
                      use_label_encoder=False, 
                      verbosity=0)

model.fit(x_train, y_train)
results = evaluate_model(model, x_test, y_test)
print(results)

{'auc_roc_curve': 0.7463180392129056}


# Logistic Regression Baseline

In [9]:
model = LogisticRegression(random_state=seed, solver='liblinear')
model.fit(x_train, y_train)
results = evaluate_model(model, x_test, y_test)
print(results)

{'auc_roc_curve': 0.7482452812440868}


# Feature Engineering

Here wee will experiment creating synthetic features using central tendency statistics.

In [10]:
geomean = lambda x, axis : np.exp(np.mean(np.log(x), axis=axis))
harmonic_mean = lambda x, axis : len(x) / np.sum(1.0/x, axis=axis) 

funcs = {'mean' : np.mean, 
         'std' : np.std, 
         'var' : np.var, 
         'geo_mean' : geomean, 
         'harmonic_mean' : harmonic_mean, 
         'median' : np.median}

<h3>Feature Engineering - XGBoost</h3>

In [11]:
results, names = list(), list()

params = {'colsample_bytree': 0.1,
          'eta': 0.12,
          'gamma': 5, 
          'max_depth': 2,
          'min_child_weight': 9,
          'n_estimators': 11000, 
          'subsample': 0.9}

for key in funcs.keys():
    x_train = train_df.drop(['id', 'target'], axis=1)
    x_train[key] = funcs[key](x_train, axis=1)
    y_train = train_df['target']
    x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size = 0.2, random_state = seed)       
    model = XGBClassifier(**params, 
                      random_state=seed, 
                      tree_method='gpu_hist', 
                      predictor='gpu_predictor',
                      use_label_encoder=False, 
                      verbosity=0)

    model.fit(x_train, y_train)
    result = evaluate_model(model, x_test, y_test)
    names.append(key)
    results.append(result['auc_roc_curve'])
    
for name, score in zip(names, results):
    print('>%s: %f' % (name, score))

>mean: 0.746157
>std: 0.746071
>var: 0.746071
>geo_mean: 0.746189
>harmonic_mean: 0.746146
>median: 0.746140


<h3>Feature Engineering - Logistic Regression</h3>

In [12]:
results, names = list(), list()

for key in funcs.keys():
    x_train = train_df.drop(['id', 'target'], axis=1)
    x_train[key] = funcs[key](x_train, axis=1)
    y_train = train_df['target']
    x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size = 0.2, random_state = seed)       
    model = LogisticRegression(random_state=seed, solver='liblinear')
    
    model.fit(x_train, y_train)
    result = evaluate_model(model, x_test, y_test)
    names.append(key)
    results.append(result['auc_roc_curve'])

for name, score in zip(names, results):
    print('>%s: %f' % (name, score))

>mean: 0.748249
>std: 0.748224
>var: 0.746519
>geo_mean: 0.748245
>harmonic_mean: 0.721463
>median: 0.748217


# Grid Search Hyperparameters

In [13]:
x_train = train_df.drop(['id', 'target'], axis=1).values
y_train = train_df['target'].values

<h3>Grid Search Hyperparameters - XGBoost<h3>

In [14]:
params = {'n_estimators' : [10000, 11000],
          'max_depth' : [2, 3],
          'subsample' : [0.8, 0.9, 1.0],
          'eta' : [0.11, 0.12],
          'colsample_bytree' : [0.3, 0.4],
          'min_child_weight': [5],
          'gamma': [5]
         }

model = XGBClassifier(random_state=seed, tree_method='gpu_hist', predictor='gpu_predictor', use_label_encoder=False, verbosity=0)
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=1, random_state=1)
search = GridSearchCV(model, param_grid=params, scoring='roc_auc', refit='roc_auc', n_jobs=-1, cv=cv)

In [15]:
result = search.fit(x_train, y_train)
print('Best Score: %s' % result.best_score_)
print('Best Hyperparameters: %s' % result.best_params_)

Best Score: 0.7466454334143676
Best Hyperparameters: {'colsample_bytree': 0.3, 'eta': 0.11, 'gamma': 5, 'max_depth': 2, 'min_child_weight': 5, 'n_estimators': 11000, 'subsample': 0.9}


<h3>Grid Search Hyperparameters - Logistic Regression<h3>

In [16]:
space = dict()
space['solver'] = ['liblinear', 'newton-cg', 'lbfgs']
space['penalty'] = ['none', 'l1', 'l2', 'elasticnet']
space['C'] = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]
model = LogisticRegression(random_state=seed, verbose=0)
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=1, random_state=1)
search = GridSearchCV(model, space, scoring='roc_auc', n_jobs=-1, cv=cv)

In [17]:
result = search.fit(x_train, y_train)
print('Best Score: %s' % result.best_score_)
print('Best Hyperparameters: %s' % result.best_params_)

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py", line 1304, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py", line 454, in _check_solver
    "penalty='none' is not supported for the liblinear solver"
ValueError: penalty='none' is not supported for the liblinear solver

  FitFailedWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:1321: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters


Best Score: 0.7490210060321131
Best Hyperparameters: {'C': 1e-05, 'penalty': 'none', 'solver': 'newton-cg'}


# Ensemble Models

In [18]:
x_train = train_df.drop(['id', 'target'], axis=1).values
y_train = train_df['target'].values 
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size = 0.2, random_state = seed)

<h4>Fitting XGBoost</h4>

In [19]:
params = {'colsample_bytree': 0.1,
          'eta': 0.12,
          'gamma': 5, 
          'max_depth': 2,
          'min_child_weight': 9,
          'n_estimators': 11000, 
          'subsample': 0.9}

xgboost_model = XGBClassifier(**params, 
                      random_state=seed, 
                      tree_method='gpu_hist', 
                      predictor='gpu_predictor',
                      use_label_encoder=False, 
                      verbosity=0)

xgboost_model.fit(x_train, y_train)
results = evaluate_model(xgboost_model, x_test, y_test)
print(results)

{'auc_roc_curve': 0.7463180392129056}


<h4>Fitting Logistic Regression</h4>

In [20]:
logistic_reg_model = LogisticRegression(random_state=seed, solver='liblinear')
logistic_reg_model.fit(x_train, y_train)
results = evaluate_model(logistic_reg_model, x_test, y_test)
print(results)

{'auc_roc_curve': 0.7482452812440868}


<h4>Ensembling both models</h4>

In [21]:
y_pred_prob_xgboost = xgboost_model.predict_proba(x_test)[:, 1]
y_pred_prob_logistic_reg = logistic_reg_model.predict_proba(x_test)[:, 1]
y_pred_prob = np.mean([y_pred_prob_xgboost, y_pred_prob_logistic_reg], axis=0)
auc_roc = roc_auc_score(y_test, y_pred_prob)
print({'auc_roc_curve' : auc_roc})

{'auc_roc_curve': 0.7485878280531844}


# Neural Nets

In [22]:
x_train = train_df.drop(['id', 'target'], axis=1).values
y_train = train_df['target'].values 
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size = 0.2, random_state = seed)

In [23]:
# https://keras.io/examples/vision/gradient_centralization/

class GCAdam(tf.keras.optimizers.Adam):
    def get_gradients(self, loss, params):
        # We here just provide a modified get_gradients() function since we are
        # trying to just compute the centralized gradients.

        grads = []
        gradients = super().get_gradients()
        for grad in gradients:
            grad_len = len(grad.shape)
            if grad_len > 1:
                axis = list(range(grad_len - 1))
                grad -= tf.reduce_mean(grad, axis=axis, keep_dims=True)
            grads.append(grad)

        return grads

In [24]:
class GCRMSprop(tf.keras.optimizers.RMSprop):
    def get_gradients(self, loss, params):
        # We here just provide a modified get_gradients() function since we are
        # trying to just compute the centralized gradients.

        grads = []
        gradients = super().get_gradients()
        for grad in gradients:
            grad_len = len(grad.shape)
            if grad_len > 1:
                axis = list(range(grad_len - 1))
                grad -= tf.reduce_mean(grad, axis=axis, keep_dims=True)
            grads.append(grad)

        return grads

In [25]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(256, activation='swish'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(128, activation='swish'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(64, activation='swish'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.compile(optimizer=GCRMSprop(learning_rate=1e-4), loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=[tf.keras.metrics.AUC(from_logits=True)])
model.fit(x_train, y_train, batch_size=128, epochs=25)

2021-11-23 07:22:03.168546: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-23 07:22:03.170082: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-23 07:22:03.170858: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-23 07:22:03.171926: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Epoch 1/25
3750/3750 [==============================] - 15s 4ms/step - loss: 0.8819 - auc: 0.5459
Epoch 2/25
3750/3750 [==============================] - 13s 3ms/step - loss: 0.6543 - auc: 0.6666
Epoch 3/25
3750/3750 [==============================] - 13s 3ms/step - loss: 0.6280 - auc: 0.7094
Epoch 4/25
3750/3750 [==============================] - 13s 4ms/step - loss: 0.6107 - auc: 0.7280
Epoch 5/25
3750/3750 [==============================] - 13s 3ms/step - loss: 0.5992 - auc: 0.7373
Epoch 6/25
1152/3750 [========>.....................] - ETA: 8s - loss: 0.5944 - auc: 0.7403

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py", line 1417, in fit
    for class_, warm_start_coef_ in zip(classes_, warm_start_coef))
  File "/opt/conda/lib/python3.7/site-packages/joblib/parallel.py", line 1041, in __call__
    if self.dispatch_one_batch(iterator):
  File "/opt/conda/lib/python3.7/site-packages/joblib/parallel.py", line 859, in dispatch_one_batch
    self._dispatch(tasks)
  File "/opt/conda/lib/python3.7/site-packages/joblib/parallel.py", line 777, in _dispatch
    job = self._backend.apply_async(batch, callback=cb)
  File "/op

1406/3750 [==========>...................] - ETA: 8s - loss: 0.5945 - auc: 0.7398

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py", line 1417, in fit
    for class_, warm_start_coef_ in zip(classes_, warm_start_coef))
  File "/opt/conda/lib/python3.7/site-packages/joblib/parallel.py", line 1041, in __call__
    if self.dispatch_one_batch(iterator):
  File "/opt/conda/lib/python3.7/site-packages/joblib/parallel.py", line 859, in dispatch_one_batch
    self._dispatch(tasks)
  File "/opt/conda/lib/python3.7/site-packages/joblib/parallel.py", line 777, in _dispatch
    job = self._backend.apply_async(batch, callback=cb)
  File "/op

3750/3750 [==============================] - 13s 4ms/step - loss: 0.5924 - auc: 0.7413
Epoch 7/25
3750/3750 [==============================] - 13s 4ms/step - loss: 0.5874 - auc: 0.7447
Epoch 8/25
3750/3750 [==============================] - 13s 3ms/step - loss: 0.5841 - auc: 0.7463
Epoch 9/25
3750/3750 [==============================] - 14s 4ms/step - loss: 0.5819 - auc: 0.7473
Epoch 10/25
3750/3750 [==============================] - 13s 3ms/step - loss: 0.5801 - auc: 0.7483
Epoch 11/25
3750/3750 [==============================] - 13s 3ms/step - loss: 0.5790 - auc: 0.7488
Epoch 12/25
3750/3750 [==============================] - 14s 4ms/step - loss: 0.5779 - auc: 0.7493
Epoch 13/25
3750/3750 [==============================] - 13s 3ms/step - loss: 0.5771 - auc: 0.7498
Epoch 14/25
3750/3750 [==============================] - 14s 4ms/step - loss: 0.5762 - auc: 0.7509
Epoch 15/25
3750/3750 [==============================] - 13s 3ms/step - loss: 0.5754 - auc: 0.7511
Epoch 16/25
3750/3750 [==

In [26]:
results =model.evaluate(x_test, y_test)[1]
print(results)

3750/3750 [==============================] - 9s 2ms/step - loss: 0.5763 - auc: 0.7497
0.7496902346611023


In [27]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(256, activation='swish'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(128, activation='swish'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(64, activation='swish'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.compile(optimizer=GCAdam(), loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=[tf.keras.metrics.AUC(from_logits=True)])
model.fit(x_train, y_train, batch_size=128, epochs=25)

Epoch 1/25
3750/3750 [==============================] - 12s 3ms/step - loss: 0.6848 - auc_1: 0.6682
Epoch 2/25
3750/3750 [==============================] - 11s 3ms/step - loss: 0.5933 - auc_1: 0.7390
Epoch 3/25
3750/3750 [==============================] - 13s 3ms/step - loss: 0.5847 - auc_1: 0.7435
Epoch 4/25
3750/3750 [==============================] - 11s 3ms/step - loss: 0.5829 - auc_1: 0.7442
Epoch 5/25
3750/3750 [==============================] - 11s 3ms/step - loss: 0.5811 - auc_1: 0.7451
Epoch 6/25
3750/3750 [==============================] - 13s 4ms/step - loss: 0.5801 - auc_1: 0.7459
Epoch 7/25
3750/3750 [==============================] - 11s 3ms/step - loss: 0.5804 - auc_1: 0.7453
Epoch 8/25
3750/3750 [==============================] - 12s 3ms/step - loss: 0.5797 - auc_1: 0.7457
Epoch 9/25
3750/3750 [==============================] - 13s 3ms/step - loss: 0.5781 - auc_1: 0.7472
Epoch 10/25
3750/3750 [==============================] - 11s 3ms/step - loss: 0.5782 - auc_1: 0.7475

In [28]:
results =model.evaluate(x_test, y_test)[1]
print(results)

3750/3750 [==============================] - 8s 2ms/step - loss: 0.5710 - auc_1: 0.7500
0.7500081658363342


In [29]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(256, activation='swish'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(128, activation='swish'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(64, activation='swish'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=[tf.keras.metrics.AUC(from_logits=True)])
model.fit(x_train, y_train, batch_size=128, epochs=25)

Epoch 1/25
3750/3750 [==============================] - 12s 3ms/step - loss: 0.6985 - auc_2: 0.6732
Epoch 2/25
3750/3750 [==============================] - 12s 3ms/step - loss: 0.5928 - auc_2: 0.7398
Epoch 3/25
3750/3750 [==============================] - 14s 4ms/step - loss: 0.5845 - auc_2: 0.7442
Epoch 4/25
3750/3750 [==============================] - 12s 3ms/step - loss: 0.5828 - auc_2: 0.7445
Epoch 5/25
3750/3750 [==============================] - 12s 3ms/step - loss: 0.5811 - auc_2: 0.7456
Epoch 6/25
3750/3750 [==============================] - 12s 3ms/step - loss: 0.5808 - auc_2: 0.7454
Epoch 7/25
3750/3750 [==============================] - 11s 3ms/step - loss: 0.5804 - auc_2: 0.7458
Epoch 8/25
3750/3750 [==============================] - 12s 3ms/step - loss: 0.5791 - auc_2: 0.7466
Epoch 9/25
3750/3750 [==============================] - 14s 4ms/step - loss: 0.5795 - auc_2: 0.7464
Epoch 10/25
3750/3750 [==============================] - 11s 3ms/step - loss: 0.5781 - auc_2: 0.7474

In [30]:
results = model.evaluate(x_test, y_test)[1]
print(results)

3750/3750 [==============================] - 9s 2ms/step - loss: 0.5798 - auc_2: 0.7477
0.747651994228363


In [31]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(256, activation='swish'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(128, activation='swish'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(64, activation='swish'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-4), 
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.AUC(from_logits=True)])
model.fit(x_train, y_train, batch_size=128, epochs=35)

Epoch 1/35
3750/3750 [==============================] - 13s 3ms/step - loss: 0.9054 - auc_3: 0.5354
Epoch 2/35
3750/3750 [==============================] - 13s 3ms/step - loss: 0.6609 - auc_3: 0.6527
Epoch 3/35
3750/3750 [==============================] - 13s 4ms/step - loss: 0.6336 - auc_3: 0.7024
Epoch 4/35
3750/3750 [==============================] - 13s 3ms/step - loss: 0.6161 - auc_3: 0.7230
Epoch 5/35
3750/3750 [==============================] - 15s 4ms/step - loss: 0.6030 - auc_3: 0.7345
Epoch 6/35
3750/3750 [==============================] - 14s 4ms/step - loss: 0.5945 - auc_3: 0.7402
Epoch 7/35
3750/3750 [==============================] - 13s 3ms/step - loss: 0.5890 - auc_3: 0.7438
Epoch 8/35
3750/3750 [==============================] - 13s 4ms/step - loss: 0.5853 - auc_3: 0.7455
Epoch 9/35
3750/3750 [==============================] - 13s 3ms/step - loss: 0.5825 - auc_3: 0.7469
Epoch 10/35
3750/3750 [==============================] - 14s 4ms/step - loss: 0.5810 - auc_3: 0.7473

In [32]:
results = model.evaluate(x_test, y_test)[1]
print(results)

3750/3750 [==============================] - 9s 2ms/step - loss: 0.5710 - auc_3: 0.7513
0.7512932419776917


In [33]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(256, activation='swish'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(128, activation='swish'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(64, activation='swish'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), 
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.AUC(from_logits=True)])
model.fit(x_train, y_train, batch_size=1024, epochs=40)

Epoch 1/40
469/469 [==============================] - 2s 4ms/step - loss: 1.2832 - auc_4: 0.5025
Epoch 2/40
469/469 [==============================] - 2s 4ms/step - loss: 0.7226 - auc_4: 0.5086
Epoch 3/40
469/469 [==============================] - 2s 4ms/step - loss: 0.6981 - auc_4: 0.5179
Epoch 4/40
469/469 [==============================] - 2s 4ms/step - loss: 0.6923 - auc_4: 0.5311
Epoch 5/40
469/469 [==============================] - 2s 4ms/step - loss: 0.6871 - auc_4: 0.5506
Epoch 6/40
469/469 [==============================] - 2s 4ms/step - loss: 0.6804 - auc_4: 0.5806
Epoch 7/40
469/469 [==============================] - 2s 4ms/step - loss: 0.6680 - auc_4: 0.6282
Epoch 8/40
469/469 [==============================] - 2s 4ms/step - loss: 0.6547 - auc_4: 0.6635
Epoch 9/40
469/469 [==============================] - 2s 4ms/step - loss: 0.6432 - auc_4: 0.6868
Epoch 10/40
469/469 [==============================] - 2s 4ms/step - loss: 0.6346 - auc_4: 0.7009
Epoch 11/40
469/469 [========

In [34]:
results = model.evaluate(x_test, y_test)[1]
print(results)

3750/3750 [==============================] - 8s 2ms/step - loss: 0.5749 - auc_4: 0.7493
0.7492884993553162


In [35]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(256, activation='swish'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(128, activation='swish'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(64, activation='swish'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), 
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.AUC(from_logits=True)])
history = model.fit(x_train, y_train, batch_size=128, epochs=35)

Epoch 1/35
3750/3750 [==============================] - 12s 3ms/step - loss: 0.9414 - auc_5: 0.5161
Epoch 2/35
3750/3750 [==============================] - 12s 3ms/step - loss: 0.6750 - auc_5: 0.6072
Epoch 3/35
3750/3750 [==============================] - 11s 3ms/step - loss: 0.6404 - auc_5: 0.6910
Epoch 4/35
3750/3750 [==============================] - 11s 3ms/step - loss: 0.6196 - auc_5: 0.7193
Epoch 5/35
3750/3750 [==============================] - 12s 3ms/step - loss: 0.6057 - auc_5: 0.7325
Epoch 6/35
3750/3750 [==============================] - 11s 3ms/step - loss: 0.5972 - auc_5: 0.7386
Epoch 7/35
3750/3750 [==============================] - 11s 3ms/step - loss: 0.5909 - auc_5: 0.7423
Epoch 8/35
3750/3750 [==============================] - 12s 3ms/step - loss: 0.5861 - auc_5: 0.7452
Epoch 9/35
3750/3750 [==============================] - 11s 3ms/step - loss: 0.5831 - auc_5: 0.7465
Epoch 10/35
3750/3750 [==============================] - 11s 3ms/step - loss: 0.5810 - auc_5: 0.7479

In [36]:
results = model.evaluate(x_test, y_test)[1]
print(results)

3750/3750 [==============================] - 9s 2ms/step - loss: 0.5712 - auc_5: 0.7510
0.7509885430335999


In [37]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(256, activation='selu'))
model.add(tf.keras.layers.Dense(128, activation='selu'))
model.add(tf.keras.layers.Dense(64, activation='selu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.AUC(from_logits=True)])
history = model.fit(x_train, y_train, batch_size=128, epochs=35)
results = model.evaluate(x_test, y_test)[1]
print(results)

Epoch 1/35
3750/3750 [==============================] - 11s 3ms/step - loss: 0.6698 - auc_6: 0.7016
Epoch 2/35
3750/3750 [==============================] - 11s 3ms/step - loss: 0.5940 - auc_6: 0.7397
Epoch 3/35
3750/3750 [==============================] - 10s 3ms/step - loss: 0.5847 - auc_6: 0.7450
Epoch 4/35
3750/3750 [==============================] - 10s 3ms/step - loss: 0.5811 - auc_6: 0.7454
Epoch 5/35
3750/3750 [==============================] - 11s 3ms/step - loss: 0.5796 - auc_6: 0.7464
Epoch 6/35
3750/3750 [==============================] - 10s 3ms/step - loss: 0.5781 - auc_6: 0.7473
Epoch 7/35
3750/3750 [==============================] - 10s 3ms/step - loss: 0.5775 - auc_6: 0.7472
Epoch 8/35
3750/3750 [==============================] - 11s 3ms/step - loss: 0.5765 - auc_6: 0.7489
Epoch 9/35
3750/3750 [==============================] - 10s 3ms/step - loss: 0.5771 - auc_6: 0.7485
Epoch 10/35
3750/3750 [==============================] - 11s 3ms/step - loss: 0.5754 - auc_6: 0.7497

# Keras Tuner

In [38]:
import keras_tuner as kt
from tensorflow import keras

In [39]:
x_train = train_df.drop(['id', 'target'], axis=1).values
y_train = train_df['target'].values

In [40]:
def model_builder(hp):
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    hp_dropout = hp.Choice('dropout', values=[0., 0.1, 0.25, 0.5])
    hp_activation = hp.Choice("activation", ["relu", "swish", "softplus", "selu"]),
    model = tf.keras.Sequential()
    
    model.add(tf.keras.layers.Dense(units=256, activation=hp_activation[0]))
    model.add(tf.keras.layers.Dropout(hp_dropout))
    model.add(tf.keras.layers.Dense(units=128, activation=hp_activation[0]))
    model.add(tf.keras.layers.Dropout(hp_dropout))
    model.add(tf.keras.layers.Dense(units=64, activation=hp_activation[0]))
    model.add(tf.keras.layers.Dropout(hp_dropout))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate), 
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), 
                  metrics=[tf.keras.metrics.AUC(name='auc', from_logits=True)])
    return model

In [41]:
# tuner = kt.Hyperband(model_builder,
                     #objective='val_auc',
                     objective = kt.Objective("val_auc", direction="max"),
                     max_epochs=25,
                     factor=3,
                     directory='my_dir3',
                     project_name='tbl_plyg_ser_nov_2021')

# stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
# tuner.search(x_train, y_train, epochs=25, validation_split=0.2, callbacks=[stop_early])
# best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]


# print(f"""
# The hyperparameter search is complete. The optimal learning rate for the optimizer
# is {best_hps.get('learning_rate')},  and the optimal dropout rate is {best_hps.get('dropout')}, and the optimal
# activation is {best_hps.get('activation')}.
# """)

IndentationError: unexpected indent (4249015303.py, line 3)

# Submission

In [ ]:
x_train = train_df.drop(['id', 'target'], axis=1)
y_train = train_df['target']

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(256, activation='swish'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(128, activation='swish'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(64, activation='swish'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), 
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.AUC(from_logits=True)])
history = model.fit(x_train, y_train, batch_size=128, epochs=25)

In [ ]:
test_df = pd.read_csv('/kaggle/input/tabular-playground-series-nov-2021/test.csv', sep=',')
x_test = test_df.drop(['id'], axis=1)
target = model.predict(x_test).squeeze()
ids = test_df['id'].values
submission = pd.DataFrame({'id' : ids, 'target' : target})

In [ ]:
submission.head()

In [ ]:
submission.to_csv('submission.csv', index=False)